In [1]:
import cudf
import numpy as np
import pandas as pd
import math

%matplotlib inline
%load_ext autotime

# Read data

In [2]:
features = [
    ('srcip', 'str')
    , ('sport', 'str')
    , ('dstip', 'str')
    , ('dsport', 'str')
    , ('proto', 'str')
    , ('state', 'str')
    , ('dur', 'float64')
    , ('sbytes', 'int64')
    , ('dbytes', 'int64')
    , ('sttl', 'int64')
    , ('dttl', 'int64')
    , ('sloss', 'int64')
    , ('dloss', 'int64')
    , ('service', 'str')
    , ('sload', 'float64')
    , ('dload', 'float64')
    , ('spkts', 'int64')
    , ('dpkts', 'int64')
    , ('swin', 'int64')
    , ('dwin', 'int64')
    , ('stcpb', 'int64')
    , ('dtcpb', 'int64')
    , ('smeansz', 'int64')
    , ('dmeansz', 'int64')
    , ('trans_depth', 'int64')
    , ('res_bdy_len', 'int64')
    , ('sjit', 'float64')
    , ('djit', 'float64')
    , ('stime', 'int64')
    , ('ltime', 'int64')
    , ('sintpkt', 'float64')
    , ('dintpkt', 'float64')
    , ('tcprtt', 'float64')
    , ('synack', 'float64')
    , ('ackdat', 'float64')
    , ('is_sm_ips_ports', 'int64')
    , ('ct_state_ttl', 'int64')
    , ('ct_flw_http_mthd', 'int64')
    , ('is_ftp_login', 'int64')
    , ('ct_ftp_cmd', 'int64')
    , ('ct_srv_src', 'int64')
    , ('ct_srv_dst', 'int64')
    , ('ct_dst_ltm', 'int64')
    , ('ct_src_ ltm', 'int64')
    , ('ct_src_dport_ltm', 'int64')
    , ('ct_dst_sport_ltm', 'int64')
    , ('ct_dst_src_ltm', 'int64')
    , ('attack_cat', 'str')
    , ('label', 'int64')
]

names  = [e[0] for e in features]
dtypes = [e[1] for e in features]

temp = []
temp.append(cudf.read_csv('../data/UNSW-NB15_1.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_2.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_3.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_4.csv', names=names, dtype=dtypes))

df = cudf.concat(temp)

for data in temp:
    del data

temp = []

time: 1.44 s


In [65]:
training_df = cudf.read_csv('../data/features/UNSW_NB15_training-set.csv')
testing_df  = cudf.read_csv('../data/features/UNSW_NB15_testing-set.csv')

### ADD IDENTIFIER
training_df['test'] = 0
testing_df['test']  = 1

# training_df = training_df.drop('id')
# testing_df  = testing_df.drop('id')

df = cudf.concat([training_df, testing_df])
df = full_df.drop(['id']).reset_index().rename({'index': 'id'})

df.head()

# ### ENCODE QUANTILES
# testing_binned = encode_quantiles(testing, quantiles)

# for col in carry_over_cols:
#     testing_binned[col] = testing[col]

# testing_binned = testing_binned.reset_index().rename({'index': 'id'})

# ### ENCODE CATEGORICAL
# cols_to_encode = ['proto', 'service', 'state']

# for col in cols_to_encode:
#     le = encoders[col]
#     testing_binned[col] = testing_binned[col].astype('category')
#     testing_binned[col + '_enc'] = le.transform(testing_binned[col])
    
#     encoders[col] = le
    
# cols_reordered = [c for c in testing_binned.columns if c not in cols_to_encode + ['label']] + ['label']
# testing_binned = testing_binned[cols_reordered]

# ### ENCODE ATTACK CAT
# testing_binned = testing_binned.merge(attack_categories, on='attack_cat')
# testing_binned = testing_binned.drop('attack_cat')
# testing_binned = testing_binned.reset_index().drop('id').rename({'index': 'id'})

# del testing     ### RELEASE THE MEMORY
# testing_binned.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label,test
0,0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,0,0,0,1,1,0,Normal,0,0
1,1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,2,0,0,0,1,6,0,Normal,0,0
2,2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,3,0,0,0,2,6,0,Normal,0,0
3,3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,3,1,1,0,2,1,0,Normal,0,0
4,4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,40,0,0,0,2,39,0,Normal,0,0


time: 440 ms


In [66]:
to_keep = [
      'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes'
    , 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss'
    , 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat'
    , 'ct_srv_src', 'ct_state_ttl', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm'
    , 'ct_srv_dst', 'attack_cat', 'label', 'test']

df = df[to_keep].reset_index(drop=True).reset_index()
df = df.rename({'index': 'id'})
df['id'] = df['id'].astype('int32')
df['attack_cat'] = df['attack_cat'].fillna('Normal')
df['attack_cat'] = df['attack_cat'].str.replace(' ', '')

time: 23.8 ms


In [67]:
df.describe()

,id,dur,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,...,synack,ackdat,ct_srv_src,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_srv_dst,label,test
count,257673.000000,257673.000000,257673.000000,257673.000000,2.576730e+05,2.576730e+05,257673.000000,257673.000000,2.576730e+05,2.576730e+05,...,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000
mean,128836.000000,1.246715,19.777144,18.514703,8.572952e+03,1.438729e+04,180.000931,84.754957,7.060869e+07,6.582143e+05,...,0.023652,0.022386,9.383176,1.324978,5.238271,4.032677,8.322964,9.121049,0.639077,0.319521
std,74383.932294,5.974305,135.947152,111.985965,1.737739e+05,1.461993e+05,102.488268,112.762131,1.857313e+08,2.412372e+06,...,0.053856,0.045771,10.829706,0.992300,8.160822,5.831515,11.120754,10.874752,0.480269,0.466292
min,0.000000,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,64418.000000,0.000008,2.000000,0.000000,1.140000e+02,0.000000e+00,62.000000,0.000000,1.231800e+04,0.000000e+00,...,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000
50%,128836.000000,0.004285,4.000000,2.000000,5.280000e+02,1.780000e+02,254.000000,29.000000,7.439423e+05,1.747441e+03,...,0.000000,0.000000,5.000000,1.000000,1.000000,1.000000,3.000000,4.000000,1.000000,0.000000
75%,193254.000000,0.685777,12.000000,10.000000,1.362000e+03,1.064000e+03,254.000000,252.000000,8.000000e+07,2.210538e+04,...,0.036842,0.044665,12.000000,2.000000,4.000000,3.000000,8.000000,11.000000,1.000000,1.000000
max,257672.000000,59.999989,10646.000000,11018.000000,1.435577e+07,1.465753e+07,255.000000,254.000000,5.988000e+09,2.242273e+07,...,3.226788,2.928778,63.000000,6.000000,59.000000,46.000000,65.000000,62.000000,1.000000,1.000000


time: 341 ms


# Encode variables

## Continuous to bins

In [68]:
bin_num = 10

quantiles = cudf.DataFrame()

for i in range(1, bin_num):
    quant = i / 10
    print(f'Quantile: {quant}')
    
    quantiles['q' + str(i)] = df.quantile(q=quant)

quantiles = quantiles.dropna().reset_index()
quantiles_list = quantiles.to_pandas().to_dict('records')

Quantile: 0.1
Quantile: 0.2
Quantile: 0.3
Quantile: 0.4
Quantile: 0.5
Quantile: 0.6
Quantile: 0.7
Quantile: 0.8
Quantile: 0.9
time: 2.65 s


In [91]:
quantiles = []

for q in quantiles_list:
    if q['index'] not in ['id', 'label', 'test']:
        elements = list(q.items())
        quants = sorted(list(set([0.0] + [e[1] for e in elements[1:]])))
        quantiles.append((elements[0][1], quants))

time: 953 µs


In [92]:
def encode_quantiles(df, quantiles):
    temp_df = cudf.DataFrame()
    
    for q in quantiles:
        col_name = q[0]
        bins = q[1]

        if df[col_name].dtype == 'int64':
            bins = [int(e) for e in bins]

        temp_df[col_name + '_bin'] = df[col_name].digitize(np.array(bins))
    return temp_df

carry_over_cols = ['proto', 'service', 'state', 'attack_cat', 'label', 'test']
df_binned = encode_quantiles(df, quantiles)

for col in carry_over_cols:
    df_binned[col] = df[col]

df_binned = df_binned.reset_index().rename({'index': 'id'})
# del df
df_binned.head()

,id,dur_bin,spkts_bin,dpkts_bin,sbytes_bin,dbytes_bin,sttl_bin,dttl_bin,sload_bin,dload_bin,...,ct_src_dport_ltm_bin,ct_dst_sport_ltm_bin,ct_dst_src_ltm_bin,ct_srv_dst_bin,proto,service,state,attack_cat,label,test
0,0,6,3,2,4,1,3,3,3,3,...,2,2,2,2,tcp,-,FIN,Normal,0,0
1,1,8,4,6,5,6,3,3,2,5,...,2,2,3,6,tcp,-,FIN,Normal,0,0
2,2,10,3,5,4,6,3,3,1,4,...,2,2,4,6,tcp,-,FIN,Normal,0,0
3,3,10,4,4,5,4,3,3,1,2,...,2,2,4,2,tcp,ftp,FIN,Normal,0,0
4,4,7,4,3,5,2,4,3,2,3,...,3,2,8,9,tcp,-,FIN,Normal,0,0


time: 266 ms


## Categorical to index

In [93]:
import cuml

encoders = {}
cols_to_encode = ['proto', 'service', 'state']

for col in cols_to_encode:
    le = cuml.preprocessing.LabelEncoder()
    df_binned[col] = df_binned[col].astype('category')
    df_binned[col + '_enc'] = le.fit_transform(df_binned[col])
    
    encoders[col] = le
    
cols_reordered = [c for c in df_binned.columns if c not in cols_to_encode + ['label']] + ['label']
df_binned = df_binned[cols_reordered]
df_binned.columns

Index(['id', 'dur_bin', 'spkts_bin', 'dpkts_bin', 'sbytes_bin', 'dbytes_bin',
       'sttl_bin', 'dttl_bin', 'sload_bin', 'dload_bin', 'sloss_bin',
       'dloss_bin', 'sjit_bin', 'djit_bin', 'swin_bin', 'stcpb_bin',
       'dtcpb_bin', 'dwin_bin', 'tcprtt_bin', 'synack_bin', 'ackdat_bin',
       'ct_srv_src_bin', 'ct_state_ttl_bin', 'ct_src_dport_ltm_bin',
       'ct_dst_sport_ltm_bin', 'ct_dst_src_ltm_bin', 'ct_srv_dst_bin',
       'attack_cat', 'test', 'proto_enc', 'service_enc', 'state_enc', 'label'],
      dtype='object')

time: 141 ms


# Explode to COO format

In [94]:
### Attack_cat encoding
attack_cat = [
      (0, 'Normal')
    , (1, 'Reconnaissance')
    , (2, 'Shellcode')
    , (3, 'Analysis')
    , (4, 'Backdoor')
    , (5, 'Backdoors')
    , (6, 'DoS')
    , (7, 'Exploits')
    , (8, 'Generic')
    , (9, 'Fuzzers')
    , (10, 'Worms')
]

attack_cat = {
      'attack_id':  [e[0] for e in attack_cat]
    , 'attack_cat': [e[1] for e in attack_cat]
}

attack_categories = cudf.DataFrame(attack_cat)
df_binned = df_binned.merge(attack_categories, on='attack_cat')
df_binned = df_binned.drop('attack_cat')
df_binned.head()

,id,dur_bin,spkts_bin,dpkts_bin,sbytes_bin,dbytes_bin,sttl_bin,dttl_bin,sload_bin,dload_bin,...,ct_src_dport_ltm_bin,ct_dst_sport_ltm_bin,ct_dst_src_ltm_bin,ct_srv_dst_bin,test,proto_enc,service_enc,state_enc,label,attack_id
0,48480,10,6,5,9,4,4,3,4,1,...,4,3,3,4,0,113,5,4,1,9
1,48481,8,4,4,6,4,4,3,2,3,...,2,2,3,4,0,113,0,4,1,9
2,48482,2,2,1,1,1,4,1,9,1,...,3,2,6,6,0,119,0,5,1,9
3,48483,2,2,1,1,1,4,1,9,1,...,4,2,6,6,0,119,0,5,1,9
4,48484,2,2,1,1,1,4,1,9,1,...,4,2,6,6,0,119,0,5,1,9


time: 146 ms


In [205]:
training_df = df_binned.query('test == 0')
training_df = training_df.drop(['test', 'id']).reset_index().rename({'index': 'id'})

# training_df.head()
df_exploded = cudf.melt(training_df, id_vars=['id', 'label', 'attack_id']).sort_values('id').reset_index(drop=True)
df_exploded['variable'] = df_exploded['variable'].astype('str')
df_exploded['value'] = df_exploded['value'].astype('str')
df_exploded['feature'] = df_exploded['variable'] + '=' + df_exploded['value']
df_exploded.drop(['variable', 'value'])

feature_encoding = df_exploded['feature'].unique().reset_index()

time: 381 ms


In [146]:
len(df_binned), len(training_df)

(257673, 175341)

time: 6.6 ms


In [206]:
feature_encoding['index'] = feature_encoding['index'].astype('int16')
feature_encoding = feature_encoding.rename({'index': 'feature_enc'})
feature_encoding.head()

,feature_enc,feature
0,0,ackdat_bin=1
1,1,ackdat_bin=2
2,2,ackdat_bin=3
3,3,ackdat_bin=4
4,4,ackdat_bin=5


time: 44.9 ms


In [207]:
df_exploded = df_exploded.merge(feature_encoding, on='feature')[['id', 'feature_enc', 'label', 'attack_id']]
df_exploded.head()

,id,feature_enc,label,attack_id
0,581,71,0,0
1,581,87,0,0
2,581,294,0,0
3,581,289,0,0
4,581,0,0,0


time: 237 ms


# Finding frequent patterns

In [208]:
def mine_patterns(df_coo, df_binned, min_attack_rate=0.75, min_feature_count=200, max_iter=-1):
    def return_rank(frate, counts, rank):
        for i, (f, c) in enumerate(zip(frate, counts)):
            rank[i] = math.log(float(c)) * f
            
    features = df_coo['feature_enc'].unique().to_frame()
    df_coo = df_coo.merge(features, on='feature_enc')

    #### FIND FREQUENT ITEMS
    freq_items = df_coo.groupby(['feature_enc']).agg({'id': 'count', 'label': 'sum'}).reset_index()
    freq_items['attack_rate'] = freq_items['label'] / freq_items['id']

    freq_items = freq_items.apply_rows(
        return_rank
        , incols = {'label': 'counts', 'attack_rate': 'frate'}
        , outcols = {'rank': np.float64}
        , kwargs = {}
    ) 

    freq_items = freq_items.sort_values('rank', ascending=False)
    freq_items.head(5)

    freq_items = freq_items.query(f'attack_rate >= {min_attack_rate} and label > {min_feature_count}')

    if max_iter == -1:
        max_iter = len(freq_items)

    devices_checked = {}

    patterns = []
    stats    = []

    features_ordered = list(freq_items['feature_enc'].to_array())

    for i in range(max_iter):
        feature = features_ordered[i]

        ### get all the ids
        ids = df_coo.query('feature_enc == @feature')['id'].unique().to_frame()
        h = ids.hash_columns(['id']).sum()

        print(f'Iter: {i}, Feature --> {feature}, devices analyzed --> {h in devices_checked}')

        if h not in devices_checked:    
            count_ids = len(ids)
            devices_checked[h] = 1

            ### OUTPUT PATTERN
            all_features = df_coo.merge(ids, on='id').groupby('feature_enc').agg({'label': 'count'}).query('label == @count_ids').reset_index()
            all_features['pattern_id'] = i

            patterns.append(all_features[['pattern_id', 'feature_enc']])

            ### OUTPUT STATS
            ids = ids.merge(df_binned, on='id')
            ids['pattern_id'] = i
            ids = ids.groupby('pattern_id').agg({'id': 'count', 'label': 'sum'})
            ids = ids.rename({'id': 'packet_count', 'label': 'attack_count'})
            ids['attack_rate'] = ids['attack_count'] / ids['packet_count']
            ids['feature_cnt'] = all_features['feature_enc'].count()
            ids = ids.reset_index()
            stats.append(ids)
            
    patterns = cudf.concat(patterns).merge(features, on='feature_enc').sort_values(by='pattern_id')
    stats = cudf.concat(stats)
    
    return patterns, stats
    
patterns, stats = mine_patterns(df_exploded, training_df, min_attack_rate=.85, min_feature_count=200)
stats

Iter: 0, Feature --> 273, devices analyzed --> False
Iter: 1, Feature --> 62, devices analyzed --> False
Iter: 2, Feature --> 73, devices analyzed --> False
Iter: 3, Feature --> 39, devices analyzed --> False
Iter: 4, Feature --> 51, devices analyzed --> False
Iter: 5, Feature --> 222, devices analyzed --> False
Iter: 6, Feature --> 40, devices analyzed --> False
Iter: 7, Feature --> 8, devices analyzed --> False
Iter: 8, Feature --> 257, devices analyzed --> False
Iter: 9, Feature --> 80, devices analyzed --> False
Iter: 10, Feature --> 78, devices analyzed --> False
Iter: 11, Feature --> 264, devices analyzed --> False
Iter: 12, Feature --> 7, devices analyzed --> False
Iter: 13, Feature --> 20, devices analyzed --> False
Iter: 14, Feature --> 14, devices analyzed --> False
Iter: 15, Feature --> 113, devices analyzed --> False
Iter: 16, Feature --> 250, devices analyzed --> False
Iter: 17, Feature --> 256, devices analyzed --> False
Iter: 18, Feature --> 27, devices analyzed --> Fals

,pattern_id,packet_count,attack_count,attack_rate,feature_cnt
0,0,82275,76560,0.930538,15
0,1,84290,77626,0.920940,11
0,2,84372,77626,0.920045,9
0,3,85195,77807,0.913281,1
0,4,89080,80801,0.907061,1
0,5,42004,39510,0.940625,8
0,6,88187,77873,0.883044,1
0,7,22427,22261,0.992598,19
0,8,23114,22790,0.985983,15
0,9,23031,22564,0.979723,14


time: 3.62 s


In [209]:
patterns_rec.query('pattern_id == 18')

,pattern_id,feature_enc,feature,col_name,col_val
170,18,27,ct_srv_dst_bin=8,ct_srv_dst_bin,8


time: 76.8 ms


In [210]:
len(df_exploded.query('feature_enc == 27')['id'].unique())

20459

time: 24.4 ms


In [211]:
len(training_df.query('ct_srv_dst_bin == 8'))

20459

time: 52.3 ms


In [212]:
df_exploded.query('feature_enc == 27')['label'].sum(), training_df.query('ct_srv_dst_bin == 8')['label'].sum()

(19291, 19291)

time: 60.9 ms


In [213]:
training_df.query('ct_srv_dst_bin == 8').groupby('label').agg({'id': 'count'})

,id
label,
0,1168
1,19291


time: 72.4 ms


In [214]:
fff = training_df[['id', 'label']].merge(training_df.query('ct_srv_dst_bin == 8')[['id', 'label']], on=['id'], how='left')
fff['label_y'] = fff['label_y'].fillna(-1)
fff.groupby('label_y').agg({'id': 'count', 'label_x': 'sum'})

,id,label_x
label_y,,
-1,154882,100050
0,1168,0
1,19291,19291


time: 91.7 ms


In [249]:

ggg = encode_patterns(training_df, patterns_to_encode)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
time: 2.64 s


In [251]:
ggg.groupby('col_0').sum()

,id,label,pred,col_1,col_2,col_3,col_4,col_5,col_6,col_7,...,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35
col_0,,,,,,,,,,,,,,,,,,,,,
0.0,5620776516,42781,41268,2015.0,2097.0,3141.0,6805.0,2588.0,5912.0,0.0,...,5385.0,16867.0,17169.0,825.0,1105.0,0.0,0.0,0.0,0.0,0.0
1.0,9770075514,76560,82275,82275.0,82275.0,82054.0,82275.0,39416.0,82275.0,22427.0,...,25147.0,0.0,0.0,1770.0,0.0,300.0,225.0,201.0,201.0,201.0


time: 220 ms


In [218]:
ggg.query('col_0 == 1').groupby('label').agg({'id': 'count'})

,id
label,
0,288
1,20171


time: 131 ms


In [217]:
len(training_df['id'].unique()), len(training_df['id'])

(175341, 175341)

time: 108 ms


# Decode patterns (host/pandas)

In [130]:
def to_set(x):
    return set(x)

def to_list(x):
    return list(x)

def ranges(bin_no, bins):
    if bin_no == len(bins):
        return '>{0:.0f}'.format(bins[-1])
    else:
        return '<{0:,.0f}, {1:,.0f})'.format(bins[bin_no-1], bins[bin_no])

quantile_bins = pd.DataFrame(quantiles, columns=['feat', 'bins'])
patterns_rec = patterns.merge(feature_encoding, on='feature_enc')

patterns_host = patterns_rec[['pattern_id', 'feature']].to_pandas()
patterns_host['feat']    = patterns_host.apply(lambda row: row['feature'].split('='), axis = 1)
patterns_host['bin']     = patterns_host.apply(lambda row: row['feat'][1], axis = 1)
patterns_host['feat']    = patterns_host.apply(lambda row: row['feat'][0][:-4], axis = 1)
patterns_host            = patterns_host.merge(quantile_bins, on=['feat'])
patterns_host['ranges']  = patterns_host.apply(lambda row: ranges(int(row['bin']), row['bins']), axis=1)
patterns_host['feature'] = patterns_host['feat'] + '=' + patterns_host['ranges']
patterns_host            = patterns_host[['pattern_id', 'feature']].sort_values(by='pattern_id')

patterns_host_agg               = patterns_host.groupby('feature').agg({'pattern_id': to_list}).reset_index()
patterns_host_agg['pattern_id'] = patterns_host_agg.apply(lambda row: ','.join([str(e) for e in row['pattern_id']]), axis = 1)#
patterns_host_agg               = patterns_host_agg.groupby('pattern_id').agg({'feature': to_set}).reset_index()

patterns_host_agg.to_dict('records')

[{'pattern_id': '0,1,2,4,7,8,9,10,15,16,17,24,25,29,31,32,33,34,35,36',
  'feature': {'dload=<0, 1,747)'}},
 {'pattern_id': '0,1,2,5,7,8,9,10,11,15,16,17,24,25,27,29,31,32,33,34,35,36',
  'feature': {'tcprtt=<0, 0)'}},
 {'pattern_id': '0,1,2,5,7,8,9,10,11,15,16,17,24,25,28,29,31,32,33,34,35,36',
  'feature': {'synack=<0, 0)'}},
 {'pattern_id': '0,1,2,5,7,8,9,10,11,15,16,17,24,25,29,31,32,33,34,35,36',
  'feature': {'ackdat=<0, 0)',
   'dloss=<0, 1)',
   'dtcpb=<0, 622,783,533)',
   'dwin=<0, 255)'}},
 {'pattern_id': '0,1,2,6,7,8,9,10,15,16,17,24,25,29,31,32,33,34,35,36',
  'feature': {'dbytes=<0, 178)'}},
 {'pattern_id': '0,1,2,7,8,9,10,15,16,17,24,25,29,31,32,33,34,35,36',
  'feature': {'dttl=<0, 29)'}},
 {'pattern_id': '0,1,7,8,9,10,15,16,17,24,25,29,31,32,33,34,35,36',
  'feature': {'djit=<0, 21)', 'dpkts=<0, 2)'}},
 {'pattern_id': '0,5,7,8,9,10,11,15,16,17,24,25,29,31,32,33,34,35,36',
  'feature': {'sloss=<0, 2)'}},
 {'pattern_id': '0,7,15,29,31,32,33,34,35,36', 'feature': {'swin=<

time: 94.5 ms


# Testing

In [107]:
testing_binned = df_binned.query('test == 1')

time: 243 ms


In [21]:
for col in cols_to_encode:
#     print(col)
    print(testing_binned[col].unique().to_frame().merge(df_binned[col].unique().to_frame(), on=col, how='left'))

KeyError: 'proto'

time: 112 ms


## Pattern encoding

In [166]:
patterns_rec['col_name'] = patterns_rec['feature'].str.split('=')[0]
patterns_rec['col_val']  = patterns_rec['feature'].str.split('=')[1]
patterns_rec = patterns_rec.sort_values(by='pattern_id').reset_index(drop=True)
patterns_expl = patterns_rec[['pattern_id', 'col_name', 'col_val']].to_pandas().to_records()

patterns_to_encode = []

curr = 0
temp = []
for i in patterns_expl:
    if curr == i[1]:
        temp.append((i[2], i[3]))
        
    else:
        patterns_to_encode.append(temp)
        temp = [(i[2], i[3])]
    curr = i[1]

patterns_to_encode[18]

[('ct_srv_dst_bin', '8')]

time: 37.7 ms


In [284]:
def encode_patterns(binned_df, patterns_to_encode):
    subset = binned_df[['id', 'label']]
    subset['pred'] = 0

    for i, pattern in enumerate(patterns_to_encode):
#         print(i)
        q = ' and '.join([' == '.join(e) for e in pattern])
        temp = binned_df.query(q)['id'].to_frame()
        temp['enc'] = 1

        subset = subset.merge(temp, on=['id'], how='left')
        subset = subset.rename({'enc': 'col_' + str(i)})
        subset['col_' + str(i)] = subset['col_' + str(i)].fillna(0)
        subset['col_' + str(i)] = subset['col_' + str(i)].astype('float32')
        subset['pred'] = subset['pred'] + subset['col_'+str(i)]
        

    subset['pred'] = subset['pred'] > 0
    subset['pred'] = subset['pred'].astype('int8')
    
    return subset

encoded_testing = encode_patterns(testing_binned, patterns_to_encode)

time: 2.13 s


In [279]:
results = encoded_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
results

,label,pred,id
0,0,0,19240
1,0,1,17760
2,1,0,5787
3,1,1,39545


time: 136 ms


In [280]:
def calculate_metrics(df):
    ttl = df['id'].sum()
    accuracy = df.query('(label == 0 and pred == 0) or (label == 1 and pred == 1)')['id'].sum() / ttl
    
    fp = df.query('(label == 0 and pred == 1)')['id'].sum()
    tn = df.query('(label == 0 and pred == 0)')['id'].sum()
    fn = df.query('(label == 1 and pred == 0)')['id'].sum()
    tp = df.query('(label == 1 and pred == 1)')['id'].sum()
    
    fpr = fp / (fp + tn)
    fnr = fn / (fn + tp)
    
    return accuracy, (fpr + fnr) / 2

calculate_metrics(results)

(0.7139994169946072, 0.30382908320832963)

time: 33.5 ms


## Logistic Regression model

In [291]:
from cuml.linear_model import LogisticRegression
from cuml.svm import SVC

encoded_df = encode_patterns(training_df, patterns_to_encode)
encoded_df['label'] = encoded_df['label'].astype('float32')
# encoded_df.head()


time: 2.74 s


In [271]:
encoded_df.query('col_18 == 1').groupby('label').agg({'id': 'count'})

,id
label,
0.0,1168
1.0,19291


time: 189 ms


In [293]:
X = encoded_df[['col_' + str(i) for i in range(len(patterns_to_encode))]]
y = encoded_df['label']

reg = SVC()
reg.fit(X, y)

SVC(handle=<cuml.common.handle.Handle object at 0x7f78acb2a3a8>, C=1, kernel='rbf', degree=3, gamma='auto', coef0=0.0, tol=0.001, cache_size=200.0, max_iter=-1, nochange_steps=1000, verbose=False)

time: 5.82 s


In [276]:
reg.coef_.copy_to_host()

array([[0.10664105],
       [0.10536657],
       [0.10509058],
       [0.10362657],
       [0.10873049],
       [0.05494867],
       [0.09426142],
       [0.0344091 ],
       [0.03682599],
       [0.03563949],
       [0.04056603],
       [0.0875266 ],
       [0.02801672],
       [0.02839951],
       [0.03089664],
       [0.02086553],
       [0.03273295],
       [0.02102997],
       [0.0265162 ],
       [0.02383227],
       [0.02351168],
       [0.02756852],
       [0.02335461],
       [0.02598433],
       [0.01971035],
       [0.02032405],
       [0.03901863],
       [0.03504279],
       [0.03549253],
       [0.00487072],
       [0.0030746 ],
       [0.00052435],
       [0.00039909],
       [0.00037434],
       [0.00037527],
       [0.00037472]], dtype=float32)

time: 8.09 ms


## Support Vector Machines

In [294]:
encoded_testing = encoded_testing.drop('pred')
encoded_testing['pred'] = reg.predict(encoded_testing[['col_' + str(i) for i in range(len(patterns_to_encode))]])
# encoded_testing.head(30)

time: 208 ms


In [295]:
results = encoded_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
results
calculate_metrics(results)

(0.8366127386678327, 0.16515986201501953)

time: 48 ms


In [265]:
encoded_testing[['col_' + str(i) for i in range(len(patterns_to_encode))]]

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 584 ms


In [71]:
encoded_df.groupby('col_0').agg({'id': 'count', 'label': 'sum'})

,id,label
col_0,,
0.0,2259581,251932.0
1.0,280466,69351.0


time: 136 ms


In [104]:
sub = df_binned.query('ct_dst_sport_ltm_bin == 4')[['id', 'label']]
sub['pred'] = 1

sub['label'].sum(),sub['id'].count()
# sub

(22440, 22469)

time: 182 ms


In [98]:
fff = df_binned.merge(sub, on='id', how='left')#.head()
fff['pred'] = fff['pred'].fillna(0)

fff.groupby('pred').agg({'id': 'count', 'label_x': 'sum'})

,id,label_x
pred,,
0,2259581,59031
1,280466,262252


time: 69.4 ms
